In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Mounted at /content/gdrive/


In [39]:
%cd '/content/gdrive/My Drive/IDLSProject-main-2'

/content/gdrive/My Drive/IDLSProject-main-2


In [1]:
import pandas as pd
import os
import pickle

data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

In [2]:
import pandas as pd
import os
train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)
print(train.shape, test_sample.shape)

(30000, 502) (20000, 502)


In [3]:
from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape

((30000, 502), (10000, 502), (10000, 502))

In [4]:
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)

torch.Size([10000, 1])


In [5]:
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

class QRNNLayer(nn.Module):
    def __init__(self,batch_size,input_size,n_filters,kernel_size,embed_size,device,dropout):
        super(QRNNLayer,self).__init__()
        self.batch_size = batch_size
        self.input_size = input_size
        self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.embed_size = embed_size
        self.dropout = torch.nn.Dropout(dropout)
        self.device = device
        self.conv1 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv2 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
        self.conv3 = torch.nn.Conv1d(self.input_size,self.n_filters,self.kernel_size)
    
    def forward(self,masked_input, h, c):
        Z,F,O = self.masked_conv(masked_input)
        h, c = self.pool(c,Z,F,O)
        masked_input = h
        return masked_input,h,c
    
    def masked_conv(self,x):
        pad = torch.zeros([self.batch_size,1,self.input_size],device=self.device)
        x = torch.cat([pad,x],1).permute(0,2,1)
        Z = torch.tanh((self.conv1(x)))
        F = torch.sigmoid((self.conv2(x)))
        O = torch.sigmoid((self.conv3(x)))
        one_mask = torch.ones_like(F,device=self.device) - F
        F = 1 - self.dropout(one_mask)
        return Z.permute(0,2,1), F.permute(0,2,1), O.permute(0,2,1)
    
    def pool(self, prev_c,Z,F,O):
        c = torch.mul(F,prev_c) + torch.mul(1-F,Z)
        h = torch.mul(O,c)
        return h,c

class QRNN(nn.Module):
    def __init__(self,trial,vocab_size,embed_size,n_filters,kernel_size,batch_size,seq_len,layers,device,dropout):
        super(QRNN,self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        self.num_layer = trial.suggest_int("num_layer",1,4)
        self.embed_size = trial.suggest_int("embed_size",16,128)
        self.n_filters = trial.suggest_int("n_filters",50,200)
        # self.kernel_size = trial.suggest_int("kernel_size",2,5)
        # self.embed_size = embed_size
        # self.n_filters = n_filters
        self.kernel_size = kernel_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        # self.num_layer = layers
        self.device = device
        self.embedding = torch.nn.Embedding(vocab_size, self.embed_size)
        self.dense = torch.nn.Linear(self.seq_len*self.n_filters,1)
        self.QRNN_layers = torch.nn.ModuleList([QRNNLayer(self.batch_size,self.embed_size if l==0 else self.n_filters,
                                                         self.n_filters,self.kernel_size,self.embed_size,self.device,
                                                         dropout,) for l in range(self.num_layer)])
        
        
    def forward(self, x, target):
        x = self.embedding(x)
        h = torch.zeros([self.batch_size,self.seq_len,self.n_filters],device=self.device)
        c = torch.zeros_like(h,device=self.device)
        
        masked_input = x
        for l,layer in enumerate(self.QRNN_layers):
            masked_input,h,c = layer(masked_input,h,c)
        dense_input = h.reshape([self.batch_size,-1])
        logits = self.dense(dense_input)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy

In [35]:
data_dir = './qrnn-imdb-hyperband-trails_20/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./qrnn-imdb-hyperband-trails_20/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [36]:
import time
epochs = 10
n_filters = 64
kernel_size = 2
layers= 2
learning_rate = 0.001
print(len(word_dict))
vocab_size = 5000
print(vocab_size)
embed_dims = 32
seq_len = 500
dropout = 0.3
batch_size=50

4998
5000


In [37]:
import time
import optuna
from optuna.trial import TrialState
import torch.optim as optim
import torch.optim as optim
import numpy as np

# Define an objective function to be minimized.
def objective(trial):
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    #epochs = trial.suggest_int("epochs",5,15)
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
    #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = QRNN(trial,vocab_size, embed_dims, n_filters, kernel_size, batch_size, seq_len, layers, device, dropout).to(device)
    trial.set_user_attr(key="best_model", value=model)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    criterion = torch.nn.BCELoss()
    counter = 0
    QRNN_acc = []
    QRNN_valacc = []
    train_loss_epoch = []
    train_acc_epoch = []
    val_loss_epoch = []
    val_acc_epoch = []
    time_epoch = []
    final_val_acc = 0
    model.train()
    for e in range(epochs):
        start_time = time.time()
        train_loss = []
        train_acc = []
        for inputs, labels in train_dl:
            inputs, labels = inputs.cuda(), labels.cuda()
            model.zero_grad()
            logits, accuracy = model(inputs,labels)
            loss = criterion(logits,labels.float())
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 5)
            optimizer.step()
            train_loss.append(loss.item())
            train_acc.append(accuracy.item()*100/batch_size)
            if counter%100==0:
                print("Epoch: {}/{}".format(e,epochs),
                             "\tIteration: {}".format(counter),
                             "\tTrain Loss: {:.3f}".format(loss.item()),
                             "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
                QRNN_acc.append(accuracy.item()*100/batch_size)
            counter += 1
        train_loss_epoch.append(np.round(np.mean(train_loss), 3))
        train_acc_epoch.append(np.round(np.mean(train_acc), 3))
        print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))
        
        with torch.no_grad():
            model.eval()
            val_acc = []
            val_loss = []
            for inputs, labels in val_dl:
                inputs_val, labels_val = inputs.cuda(), labels.cuda()
                logits_val,accuracy_val = model(inputs_val,labels_val)
                loss_val = criterion(logits_val,labels_val.float())
                val_acc.append(accuracy_val.item()*100/batch_size)
                val_loss.append(loss_val.item())
            print("Val Loss: {:.3f}".format(np.mean(val_loss)), "\tVal Acc: {:.3f}".format(np.mean(val_acc)))
            val_loss_epoch.append(np.round(np.mean(val_loss), 3))
            val_acc_epoch.append(np.round(np.mean(val_acc), 3))
            QRNN_valacc.append(np.mean(val_acc))
            model.train()
            final_val_acc = np.mean(val_acc)
            end_time = time.time()-start_time
            print("Time to train epoch: {0} s".format(end_time))
            time_epoch.append(np.round(end_time, 3))
    write_to_csv(trial.number, epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
    return final_val_acc


In [38]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [39]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=20,callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-14 16:04:49,527] A new study created in memory with name: no-name-0e99f648-a883-4815-b53f-31623514f3e5


Epoch: 0/10 	Iteration: 0 	Train Loss: 0.693 	Train Accuracy: 54.00
Epoch: 0/10 	Iteration: 100 	Train Loss: 48.824 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 200 	Train Loss: 46.000 	Train Accuracy: 54.00
Epoch: 0/10 	Iteration: 300 	Train Loss: 56.000 	Train Accuracy: 44.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 500 	Train Loss: 50.000 	Train Accuracy: 50.00
	Train Loss: 49.510 	Train Acc: 50.110
Val Loss: 49.760 	Val Acc: 50.240
Time to train epoch: 25.429874181747437 s
Epoch: 1/10 	Iteration: 600 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 700 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 1/10 	Iteration: 800 	Train Loss: 46.000 	Train Accuracy: 54.00
Epoch: 1/10 	Iteration: 900 	Train Loss: 60.000 	Train Accuracy: 40.00
Epoch: 1/10 	Iteration: 1000 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 1100 	Train Loss: 50.000 	Train Accuracy: 50.00
	Train Loss: 49.911 	Train Acc: 49

[I 2022-05-14 16:09:03,677] Trial 0 finished with value: 50.24 and parameters: {'optimizer': 'Adam', 'lr': 0.21626720751825737, 'dropout': 0.49825009795037967, 'num_layer': 4, 'embed_size': 37, 'n_filters': 192}. Best is trial 0 with value: 50.24.


Val Loss: 49.760 	Val Acc: 50.240
Time to train epoch: 25.411388397216797 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.691 	Train Accuracy: 54.00
Epoch: 0/10 	Iteration: 100 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 200 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 300 	Train Loss: 60.000 	Train Accuracy: 40.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 500 	Train Loss: 50.000 	Train Accuracy: 50.00
	Train Loss: 49.767 	Train Acc: 50.110
Val Loss: 49.760 	Val Acc: 50.240
Time to train epoch: 14.600804328918457 s
Epoch: 1/10 	Iteration: 600 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 700 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 1/10 	Iteration: 800 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 1/10 	Iteration: 900 	Train Loss: 60.000 	Train Accuracy: 40.00
Epoch: 1/10 	Iteration: 1000 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 1100 	

[I 2022-05-14 16:11:31,183] Trial 1 finished with value: 50.24 and parameters: {'optimizer': 'RMSprop', 'lr': 0.018410213262972947, 'dropout': 0.377322840240922, 'num_layer': 4, 'embed_size': 21, 'n_filters': 94}. Best is trial 0 with value: 50.24.


...

Val Loss: 0.348 	Val Acc: 86.390
Time to train epoch: 6.434581995010376 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.693 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 100 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 200 	Train Loss: 48.000 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 300 	Train Loss: 60.000 	Train Accuracy: 40.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 500 	Train Loss: 50.000 	Train Accuracy: 50.00
	Train Loss: 49.668 	Train Acc: 50.180
Val Loss: 50.240 	Val Acc: 49.760
Time to train epoch: 24.9097638130188 s
Epoch: 1/10 	Iteration: 600 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 700 	Train Loss: 52.000 	Train Accuracy: 48.00
Epoch: 1/10 	Iteration: 800 	Train Loss: 52.000 	Train Accuracy: 48.00
Epoch: 1/10 	Iteration: 900 	Train Loss: 40.000 	Train Accuracy: 60.00
Epoch: 1/10 	Iteration: 1000 	Train Loss: 50.000 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 1100 	Trai

[I 2022-05-14 16:43:02,443] Trial 18 finished with value: 49.76 and parameters: {'optimizer': 'RMSprop', 'lr': 0.006047262814334825, 'dropout': 0.3720174176995784, 'num_layer': 4, 'embed_size': 90, 'n_filters': 176}. Best is trial 9 with value: 87.48.


Val Loss: 50.240 	Val Acc: 49.760
Time to train epoch: 24.96263289451599 s
Epoch: 0/10 	Iteration: 0 	Train Loss: 0.690 	Train Accuracy: 54.00
Epoch: 0/10 	Iteration: 100 	Train Loss: 0.696 	Train Accuracy: 42.00
Epoch: 0/10 	Iteration: 200 	Train Loss: 0.694 	Train Accuracy: 46.00
Epoch: 0/10 	Iteration: 300 	Train Loss: 0.695 	Train Accuracy: 44.00
Epoch: 0/10 	Iteration: 400 	Train Loss: 0.691 	Train Accuracy: 62.00
Epoch: 0/10 	Iteration: 500 	Train Loss: 0.693 	Train Accuracy: 56.00
	Train Loss: 0.693 	Train Acc: 50.170
Val Loss: 0.693 	Val Acc: 49.690
Time to train epoch: 15.040668725967407 s
Epoch: 1/10 	Iteration: 600 	Train Loss: 0.695 	Train Accuracy: 44.00
Epoch: 1/10 	Iteration: 700 	Train Loss: 0.695 	Train Accuracy: 44.00
Epoch: 1/10 	Iteration: 800 	Train Loss: 0.696 	Train Accuracy: 42.00
Epoch: 1/10 	Iteration: 900 	Train Loss: 0.694 	Train Accuracy: 38.00
Epoch: 1/10 	Iteration: 1000 	Train Loss: 0.693 	Train Accuracy: 50.00
Epoch: 1/10 	Iteration: 1100 	Train Loss: 0

[I 2022-05-14 16:45:33,024] Trial 19 finished with value: 49.66 and parameters: {'optimizer': 'SGD', 'lr': 1.0166476591232525e-05, 'dropout': 0.2818696043093479, 'num_layer': 3, 'embed_size': 111, 'n_filters': 138}. Best is trial 9 with value: 87.48.


Val Loss: 0.693 	Val Acc: 49.660
Time to train epoch: 15.105906248092651 s
Study statistics: 
  Number of finished trials:  20
  Number of complete trials:  20
Best trial:
  Value:  87.48
  Params: 
    optimizer: RMSprop
    lr: 0.00034669386244841604
    dropout: 0.18879994913780898
    num_layer: 3
    embed_size: 93
    n_filters: 57


In [40]:
data_dir = './qrnn-imdb-hyperband/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)
    
torch.save(best_model.state_dict(),"./qrnn-imdb-hyperband/qrnn-imdb-hyperband_20.pth")

In [41]:
criterion = torch.nn.BCELoss()
def test(model, test_dl, epochs):
    with torch.no_grad():
        model.eval()
        test_acc = []
        test_loss = []
        for inputs, labels in test_dl:
            input_test, labels_test = inputs.cuda(), labels.cuda()
            logits_test, accuracy_test = model(input_test, labels_test)
            loss_test = criterion(logits_test, labels_test.float())
            test_acc.append(accuracy_test.item()*100/batch_size)
            test_loss.append(loss_test.item())
        print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))

In [42]:
test(best_model, test_dl, epochs)

Test Loss: 0.325 	Test Acc: 88.030
